In [1]:
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from keras import layers
import tensorflow_datasets as tfds

epochs = 30
batch_size = 64
image_w, image_h = 32, 32

!rm -rf ./logs/imagenette/*

In [2]:
(ds_train_raw, ds_test_raw), ds_info = tfds.load(
    "imagenette",
    split=["train", "validation"],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

n_classes = ds_info.features["label"].num_classes
n = ds_info.splits["train"].num_examples


def normalize_img(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = layers.Resizing(image_h, image_w)(image)
    image = tf.reshape(image, [-1])
    label = tf.one_hot(tf.cast(label, tf.int32), n_classes)
    label = tf.cast(label, tf.float32)
    return image, label


def normalize(ds):
    ds = (
        ds.cache()
        .map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return ds


_, dims = list(
    ds_train_raw.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
    .take(1)
    .batch(1)
    .as_numpy_iterator()
)[0][0].shape

print("n: ", n, "n_classes: ", n_classes, "dims: ", dims)


n:  9469 n_classes:  10 dims:  3072


In [3]:
RandomFourierFeatures = keras.layers.experimental.RandomFourierFeatures

model_svm = keras.Sequential(
    [
        layers.Input(shape=(dims,)),
        RandomFourierFeatures(
            output_dim=2000, scale=10.0, kernel_initializer="gaussian"
        ),
        layers.Dense(units=n_classes),
    ]
)
model_svm.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.hinge,
    metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
)

modeldir = "./logs/imagenette/linear-8192-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_svm.fit(
    normalize(ds_train_raw),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=normalize(ds_test_raw),
    callbacks=[
        keras.callbacks.TensorBoard(
            log_dir=modeldir + "/log",
            histogram_freq=1,
        )
    ],
)
model_svm.save(modeldir + "/model")


Epoch 1/30
148/148 [==============================] - 9s 50ms/step - loss: 0.3514 - acc: 0.2651 - val_loss: 0.3049 - val_acc: 0.3195
Epoch 2/30
148/148 [==============================] - 6s 40ms/step - loss: 0.2606 - acc: 0.4049 - val_loss: 0.2859 - val_acc: 0.3557
Epoch 3/30
148/148 [==============================] - 6s 41ms/step - loss: 0.2257 - acc: 0.4969 - val_loss: 0.2822 - val_acc: 0.3684
Epoch 4/30
148/148 [==============================] - 6s 43ms/step - loss: 0.2058 - acc: 0.5461 - val_loss: 0.2824 - val_acc: 0.3699
Epoch 5/30
148/148 [==============================] - 7s 45ms/step - loss: 0.1928 - acc: 0.5824 - val_loss: 0.2856 - val_acc: 0.3674
Epoch 6/30
148/148 [==============================] - 6s 44ms/step - loss: 0.1835 - acc: 0.6039 - val_loss: 0.2904 - val_acc: 0.3692
Epoch 7/30
148/148 [==============================] - 6s 42ms/step - loss: 0.1764 - acc: 0.6282 - val_loss: 0.2946 - val_acc: 0.3656
Epoch 8/30
148/148 [==============================] - 6s 41ms/step - 

INFO:tensorflow:Assets written to: ./logs/imagenette/linear-8192-20230308-114934/model\assets


In [4]:
def gen_samples(psi, t=1000):
    return [
        list(
            ds_train_raw.shuffle(n)
            .take(psi)
            .map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
            .batch(psi)
            .as_numpy_iterator()
        )[0][0]
        for _ in range(t)
    ]


def _tf_ann(X, samples, p=2, soft=True):
    m_dis = None
    for i in range(samples.shape[0]):
        i_sample = samples[i : i + 1, :]
        l_dis = tf.math.reduce_sum((X - i_sample) ** p, axis=1, keepdims=True) ** (
            1 / p
        )
        if m_dis is None:
            m_dis = l_dis
        else:
            m_dis = tf.concat([m_dis, l_dis], 1)

    if soft:
        feature_map = tf.nn.softmax(-m_dis, axis=0)
    else:
        feature_map = tf.one_hot(tf.math.argmax(-m_dis, axis=1), samples.shape[0])
    # l_dis_min = tf.math.reduce_sum(m_dis * feature_map, axis=0)
    return feature_map


In [5]:
class IsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, soft=True, **kwargs):
        super(IsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p
        self.soft = soft

    def call(self, inputs):
        return _tf_ann(inputs, self.samples, self.p, self.soft)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
                "soft": self.soft,
            }
        )
        return config


def build_model(t_samples, p=2, soft=True):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        IsolationEncodingLayer(t_samples[i], p=p, soft=soft, name="ann_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=n_classes, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


In [6]:
class FlexibleIsolationEncodingLayer(layers.Layer):
    def __init__(self, samples, p=2, **kwargs):
        super(FlexibleIsolationEncodingLayer, self).__init__(**kwargs)
        self.samples = samples
        self.p = p

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.dimential_weights = self.add_weight(
            name="dimential_weights",
            shape=(
                1,
                self.samples.shape[0],
            ),
            initializer="ones",
            trainable=True,
        )
        super(FlexibleIsolationEncodingLayer, self).build(input_shape)

    def call(self, inputs):
        return _tf_ann(inputs, self.samples, self.p) * self.dimential_weights

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "samples": self.samples,
                "p": self.p,
            }
        )
        return config


def build_flex_model(t_samples, p=2):
    t = len(t_samples)
    if t <= 0:
        raise ValueError("t <= 0")
    _, dims = t_samples[0].shape

    inputs = keras.Input(name="inputs_x", shape=(dims,))
    lambdas = [
        FlexibleIsolationEncodingLayer(t_samples[i], p=p, name="ann_flex_{}".format(i))(
            inputs
        )
        for i in range(t)
    ]
    concatenated = layers.Concatenate(axis=1, name="concatenated")(lambdas)
    outputs = layers.Dense(units=n_classes, name="outputs_y")(concatenated)

    model = keras.Model(name="isolation_encoding", inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=1e-3),
        loss=keras.losses.hinge,
        metrics=[keras.metrics.CategoricalAccuracy(name="acc")],
    )
    return model


In [7]:
t_samples = gen_samples(psi=20, t=50)


model_hard_20_50 = build_model(t_samples, soft=False)
modeldir = "./logs/imagenette/hard-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_20_50.fit(
    normalize(ds_train_raw),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=normalize(ds_test_raw),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_20_50.save(modeldir + "/model")


Epoch 1/30
148/148 [==============================] - 49s 237ms/step - loss: 0.3014 - acc: 0.2111 - val_loss: 0.2202 - val_acc: 0.2591
Epoch 2/30
148/148 [==============================] - 26s 177ms/step - loss: 0.2127 - acc: 0.2990 - val_loss: 0.2124 - val_acc: 0.2803
Epoch 3/30
148/148 [==============================] - 26s 174ms/step - loss: 0.2057 - acc: 0.3433 - val_loss: 0.2088 - val_acc: 0.2991
Epoch 4/30
148/148 [==============================] - 26s 173ms/step - loss: 0.2014 - acc: 0.3733 - val_loss: 0.2064 - val_acc: 0.3141
Epoch 5/30
148/148 [==============================] - 26s 176ms/step - loss: 0.1986 - acc: 0.4006 - val_loss: 0.2052 - val_acc: 0.3162
Epoch 6/30
148/148 [==============================] - 26s 176ms/step - loss: 0.1967 - acc: 0.4152 - val_loss: 0.2042 - val_acc: 0.3213
Epoch 7/30
148/148 [==============================] - 26s 175ms/step - loss: 0.1954 - acc: 0.4257 - val_loss: 0.2037 - val_acc: 0.3185
Epoch 8/30
148/148 [==============================] - 2

INFO:tensorflow:Assets written to: ./logs/imagenette/hard-20x50-20230308-115629/model\assets


In [8]:
model_soft_20_50 = build_model(t_samples, soft=True)
modeldir = "./logs/imagenette/soft-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_20_50.fit(
    normalize(ds_train_raw),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=normalize(ds_test_raw),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_20_50.save(modeldir + "/model")


Epoch 1/30
148/148 [==============================] - 56s 275ms/step - loss: 0.6282 - acc: 0.2211 - val_loss: 0.5057 - val_acc: 0.2576
Epoch 2/30
148/148 [==============================] - 27s 181ms/step - loss: 0.4534 - acc: 0.2761 - val_loss: 0.4300 - val_acc: 0.2836
Epoch 3/30
148/148 [==============================] - 27s 182ms/step - loss: 0.3924 - acc: 0.3003 - val_loss: 0.3803 - val_acc: 0.2996
Epoch 4/30
148/148 [==============================] - 26s 178ms/step - loss: 0.3477 - acc: 0.3209 - val_loss: 0.3397 - val_acc: 0.3118
Epoch 5/30
148/148 [==============================] - 27s 182ms/step - loss: 0.3105 - acc: 0.3390 - val_loss: 0.3048 - val_acc: 0.3340
Epoch 6/30
148/148 [==============================] - 27s 182ms/step - loss: 0.2792 - acc: 0.3596 - val_loss: 0.2756 - val_acc: 0.3366
Epoch 7/30
148/148 [==============================] - 27s 179ms/step - loss: 0.2541 - acc: 0.3736 - val_loss: 0.2525 - val_acc: 0.3409
Epoch 8/30
148/148 [==============================] - 2

INFO:tensorflow:Assets written to: ./logs/imagenette/soft-20x50-20230308-121050/model\assets


In [9]:
model_flex_20_50 = build_flex_model(t_samples)
# model_flex_20_50.summary()
modeldir = "./logs/imagenette/flex-20x50-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_flex_20_50.fit(
    normalize(ds_train_raw),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=normalize(ds_test_raw),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_flex_20_50.save(modeldir + "/model")


Epoch 1/30
148/148 [==============================] - 61s 280ms/step - loss: 0.6235 - acc: 0.2239 - val_loss: 0.4984 - val_acc: 0.2632
Epoch 2/30
148/148 [==============================] - 30s 205ms/step - loss: 0.4443 - acc: 0.2846 - val_loss: 0.4234 - val_acc: 0.2833
Epoch 3/30
148/148 [==============================] - 30s 206ms/step - loss: 0.3848 - acc: 0.3100 - val_loss: 0.3746 - val_acc: 0.2983
Epoch 4/30
148/148 [==============================] - 30s 206ms/step - loss: 0.3412 - acc: 0.3287 - val_loss: 0.3346 - val_acc: 0.3149
Epoch 5/30
148/148 [==============================] - 30s 206ms/step - loss: 0.3051 - acc: 0.3504 - val_loss: 0.3012 - val_acc: 0.3289
Epoch 6/30
148/148 [==============================] - 31s 207ms/step - loss: 0.2756 - acc: 0.3720 - val_loss: 0.2737 - val_acc: 0.3338
Epoch 7/30
148/148 [==============================] - 31s 210ms/step - loss: 0.2528 - acc: 0.3783 - val_loss: 0.2524 - val_acc: 0.3442
Epoch 8/30
148/148 [==============================] - 3

INFO:tensorflow:Assets written to: ./logs/imagenette/flex-20x50-20230308-122548/model\assets


In [10]:
t_samples = gen_samples(psi=100, t=10)

model_hard_100_10 = build_model(t_samples, soft=False)
modeldir = "./logs/imagenette/hard-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_hard_100_10.fit(
    normalize(ds_train_raw),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=normalize(ds_test_raw),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_hard_100_10.save(modeldir + "/model")


Epoch 1/30
148/148 [==============================] - 50s 253ms/step - loss: 0.5182 - acc: 0.1743 - val_loss: 0.2268 - val_acc: 0.2321
Epoch 2/30
148/148 [==============================] - 27s 183ms/step - loss: 0.2147 - acc: 0.2774 - val_loss: 0.2109 - val_acc: 0.2854
Epoch 3/30
148/148 [==============================] - 27s 182ms/step - loss: 0.2054 - acc: 0.3289 - val_loss: 0.2056 - val_acc: 0.3029
Epoch 4/30
148/148 [==============================] - 27s 183ms/step - loss: 0.2007 - acc: 0.3561 - val_loss: 0.2028 - val_acc: 0.3118
Epoch 5/30
148/148 [==============================] - 27s 183ms/step - loss: 0.1979 - acc: 0.3752 - val_loss: 0.2009 - val_acc: 0.3177
Epoch 6/30
148/148 [==============================] - 27s 184ms/step - loss: 0.1960 - acc: 0.3865 - val_loss: 0.1997 - val_acc: 0.3175
Epoch 7/30
148/148 [==============================] - 27s 183ms/step - loss: 0.1947 - acc: 0.3960 - val_loss: 0.1989 - val_acc: 0.3172
Epoch 8/30
148/148 [==============================] - 2

INFO:tensorflow:Assets written to: ./logs/imagenette/hard-100x10-20230308-124442/model\assets


In [11]:
model_soft_100_10 = build_model(t_samples, soft=True)
modeldir = "./logs/imagenette/soft-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_soft_100_10.fit(
    normalize(ds_train_raw),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=normalize(ds_test_raw),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_soft_100_10.save(modeldir + "/model")


Epoch 1/30
148/148 [==============================] - 56s 285ms/step - loss: 0.6286 - acc: 0.2021 - val_loss: 0.5071 - val_acc: 0.2504
Epoch 2/30
148/148 [==============================] - 29s 198ms/step - loss: 0.4541 - acc: 0.2769 - val_loss: 0.4322 - val_acc: 0.2790
Epoch 3/30
148/148 [==============================] - 31s 209ms/step - loss: 0.3938 - acc: 0.3017 - val_loss: 0.3821 - val_acc: 0.3001
Epoch 4/30
148/148 [==============================] - 31s 207ms/step - loss: 0.3490 - acc: 0.3223 - val_loss: 0.3408 - val_acc: 0.3149
Epoch 5/30
148/148 [==============================] - 32s 214ms/step - loss: 0.3115 - acc: 0.3445 - val_loss: 0.3054 - val_acc: 0.3276
Epoch 6/30
148/148 [==============================] - 30s 204ms/step - loss: 0.2804 - acc: 0.3636 - val_loss: 0.2763 - val_acc: 0.3371
Epoch 7/30
148/148 [==============================] - 30s 204ms/step - loss: 0.2556 - acc: 0.3758 - val_loss: 0.2531 - val_acc: 0.3427
Epoch 8/30
148/148 [==============================] - 3

INFO:tensorflow:Assets written to: ./logs/imagenette/soft-100x10-20230308-130005/model\assets


In [12]:
model_flex_100_10 = build_flex_model(t_samples)
# model_flex_20_50.summary()
modeldir = "./logs/imagenette/flex-100x10-" + datetime.now().strftime("%Y%m%d-%H%M%S")
model_flex_100_10.fit(
    normalize(ds_train_raw),
    epochs=epochs,
    batch_size=batch_size,
    validation_data=normalize(ds_test_raw),
    callbacks=[
        keras.callbacks.TensorBoard(log_dir=modeldir + "/log", histogram_freq=1)
    ],
)
model_flex_100_10.save(modeldir + "/model")


Epoch 1/30
148/148 [==============================] - 61s 313ms/step - loss: 0.6250 - acc: 0.2046 - val_loss: 0.4967 - val_acc: 0.2609
Epoch 2/30
148/148 [==============================] - 31s 210ms/step - loss: 0.4451 - acc: 0.2782 - val_loss: 0.4234 - val_acc: 0.2831
Epoch 3/30
148/148 [==============================] - 31s 211ms/step - loss: 0.3863 - acc: 0.3038 - val_loss: 0.3759 - val_acc: 0.3027
Epoch 4/30
148/148 [==============================] - 32s 216ms/step - loss: 0.3429 - acc: 0.3288 - val_loss: 0.3362 - val_acc: 0.3220
Epoch 5/30
148/148 [==============================] - 32s 215ms/step - loss: 0.3069 - acc: 0.3513 - val_loss: 0.3026 - val_acc: 0.3350
Epoch 6/30
148/148 [==============================] - 32s 214ms/step - loss: 0.2774 - acc: 0.3681 - val_loss: 0.2751 - val_acc: 0.3460
Epoch 7/30
148/148 [==============================] - 32s 214ms/step - loss: 0.2543 - acc: 0.3857 - val_loss: 0.2532 - val_acc: 0.3506
Epoch 8/30
148/148 [==============================] - 3

INFO:tensorflow:Assets written to: ./logs/imagenette/flex-100x10-20230308-131647/model\assets
